In [1]:
%%latex
$\dfrac{\int\psi(x)H\psi(x)dx}{\int\psi(x)\psi(x)dx}$; $\psi(x)$ is given before calculation; $\psi(x)$ not need to normalize but square-integrable and continuous

<IPython.core.display.Latex object>

In [2]:
def rayleigh(hamiltonian, wavefunction, limits):
    return (sympy.integrate(wavefunction * hamiltonian(wavefunction), (position, *limits)) 
            / sympy.integrate(wavefunction**2,(position, *limits)))

#particle in a box
import sympy
length, mass, hbar = sympy.symbols('l,m,hbar', positive=True)
position = sympy.symbols('x', real=True)
wavefunction = position * (length - position)
hamiltonian = lambda wavefunction: -hbar**2 / 2 / mass * wavefunction.diff(position, 2)
exact = hbar**2 * sympy.pi**2 / 2 / mass / length**2  #曾谨言volume1 page65 3.2.7
sympy.N(rayleigh(hamiltonian, wavefunction, [0, length]) / exact - 1)

0.0132118364233777

In [3]:
#harmonic oscillator
wavefunction = sympy.exp(-length * position**2)
angularFrequency = sympy.symbols('varpi', positive=True)
hamiltonian = (lambda wavefunction: -hbar**2 / 2 / mass * wavefunction.diff(position, 2) 
               + mass * angularFrequency**2 / 2 * position**2 * wavefunction)
ground = rayleigh(hamiltonian, wavefunction, [-sympy.oo, sympy.oo])
ground.subs(length, sympy.solveset(ground.diff(length), length).sup)

h̅⋅varpi
────────
   2    

In [4]:
basis = sympy.Matrix([
    position * (length - position),
    position**2 * (length - position)**2,
    position * (length - position) * (length / 2 - position),
    position**2 * (length - position)**2 * (length / 2 - position)
])
basis

⎡     x⋅(l - x)     ⎤
⎢                   ⎥
⎢     2        2    ⎥
⎢    x ⋅(l - x)     ⎥
⎢                   ⎥
⎢   ⎛l    ⎞         ⎥
⎢ x⋅⎜─ - x⎟⋅(l - x) ⎥
⎢   ⎝2    ⎠         ⎥
⎢                   ⎥
⎢ 2 ⎛l    ⎞        2⎥
⎢x ⋅⎜─ - x⎟⋅(l - x) ⎥
⎣   ⎝2    ⎠         ⎦

In [5]:
hamiltonian = (basis @ basis.applyfunc(lambda element: -hbar**2 / 2 / mass * element.diff(position, 2)).T
              ).applyfunc(lambda element: sympy.integrate(element, (position, 0, length)))
hamiltonian

⎡  2  3    2  5                ⎤
⎢h̅ ⋅l   h̅ ⋅l                 ⎥
⎢──────  ──────    0       0   ⎥
⎢ 6⋅m     30⋅m                 ⎥
⎢                              ⎥
⎢  2  5    2  7                ⎥
⎢h̅ ⋅l   h̅ ⋅l                 ⎥
⎢──────  ──────    0       0   ⎥
⎢ 30⋅m   105⋅m                 ⎥
⎢                              ⎥
⎢                  2  5    2  7⎥
⎢                h̅ ⋅l   h̅ ⋅l ⎥
⎢  0       0     ──────  ──────⎥
⎢                 40⋅m   280⋅m ⎥
⎢                              ⎥
⎢                  2  7    2  9⎥
⎢                h̅ ⋅l   h̅ ⋅l ⎥
⎢  0       0     ──────  ──────⎥
⎣                280⋅m   1260⋅m⎦

In [6]:
overlap = (basis @ basis.T
          ).applyfunc(lambda element: sympy.integrate(element, (position, 0, length)))
overlap

⎡ 5     7             ⎤
⎢l     l              ⎥
⎢──   ───   0      0  ⎥
⎢30   140             ⎥
⎢                     ⎥
⎢  7    9             ⎥
⎢ l    l              ⎥
⎢───  ───   0      0  ⎥
⎢140  630             ⎥
⎢                     ⎥
⎢            7     9  ⎥
⎢           l     l   ⎥
⎢ 0    0   ───   ──── ⎥
⎢          840   5040 ⎥
⎢                     ⎥
⎢            9     11 ⎥
⎢           l     l   ⎥
⎢ 0    0   ────  ─────⎥
⎣          5040  27720⎦

In [7]:
LowerInverse = overlap.cholesky().inv()
coefficient, energy = (LowerInverse @ hamiltonian @ LowerInverse.T).diagonalize()
coefficient = LowerInverse.T @ sympy.Matrix.hstack(*map(
    lambda column: coefficient[:, column] / coefficient[:, column].norm(),
    range(coefficient.shape[-1])))
energy

⎡    2                                                                       ⎤
⎢2⋅h̅ ⋅(-√133 + 14)                                                          ⎥
⎢──────────────────          0                  0                   0        ⎥
⎢        2                                                                   ⎥
⎢       l ⋅m                                                                 ⎥
⎢                                                                            ⎥
⎢                        2                                                   ⎥
⎢                    2⋅h̅ ⋅(√133 + 14)                                       ⎥
⎢        0           ─────────────────          0                   0        ⎥
⎢                            2                                               ⎥
⎢                           l ⋅m                                             ⎥
⎢                                                                            ⎥
⎢                                           2       

In [8]:
assert (coefficient.T @ overlap @ coefficient).applyfunc(sympy.cancel) == sympy.eye(len(basis))  
#wavefunction are orthonormal
wavefunction = basis.T @ coefficient
assert ((wavefunction.T @ wavefunction
        ).applyfunc(lambda element: sympy.integrate(element, (position, 0, length)).cancel()) 
        == sympy.eye(len(wavefunction)))  #wavefunction are orthonormal
wavefunction.T

⎡                                                                             
⎢            ⎛               9⋅√10                                    3⋅√10   
⎢  x⋅(l - x)⋅⎜- ─────────────────────────────── + ────────────────────────────
⎢            ⎜            _____________________                            ___
⎢            ⎜   5/2     ╱            3            5/2                    ╱   
⎢            ⎜  l   ⋅   ╱  1 + ───────────────    l   ⋅(-23 + 2⋅√133)⋅   ╱  1 
⎢            ⎜         ╱                     2                          ╱     
⎢            ⎝       ╲╱        (-23 + 2⋅√133)                         ╲╱      
⎢                                                                             
⎢                                                                             
⎢              ⎛              9⋅√10                                   3⋅√10   
⎢    x⋅(l - x)⋅⎜- ────────────────────────────── - ───────────────────────────
⎢              ⎜            ____________________    

In [9]:
numerical = [(hbar, 1), (length, 1), (mass, 1)]
import scipy.linalg
numericalEnergy, numericalCoefficient = scipy.linalg.eigh(
    sympy.matrix2numpy(hamiltonian.subs(numerical), float),
    sympy.matrix2numpy(overlap.subs(numerical), float))
numericalEnergy

array([   4.93487481,   19.75077641,   51.06512519,  100.24922359])

In [10]:
energy.subs(numerical).evalf()

⎡4.93487481065841         0                 0                 0        ⎤
⎢                                                                      ⎥
⎢       0          51.0651251893416         0                 0        ⎥
⎢                                                                      ⎥
⎢       0                 0          19.7507764050038         0        ⎥
⎢                                                                      ⎥
⎣       0                 0                 0          100.249223594996⎦

In [11]:
numericalCoefficient

array([[  -4.40399751,    0.        ,  -28.64620055,    0.        ],
       [  -4.9903486 ,    0.        ,  132.7218762 ,    0.        ],
       [   0.        ,  -16.78235216,    0.        ,  -98.98662867],
       [  -0.        ,  -71.84781643,    0.        ,  572.2568403 ]])

In [12]:
coefficient.subs(numerical).evalf()

⎡4.40399751133633  -28.6462005494646         0                  0        ⎤
⎢                                                                        ⎥
⎢4.99034859672658  132.721876195611          0                  0        ⎥
⎢                                                                        ⎥
⎢       0                  0          16.7823521557751  -98.9866286733696⎥
⎢                                                                        ⎥
⎣       0                  0          71.8478164291383  572.256840303692 ⎦